In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/Xavier/Desktop/Capstone_project/google_capstone_project.json'

In [3]:
#Imports Google Cloud client library
from google.cloud import bigquery

#Instantiates a client for BigQuery service
bigquery_client = bigquery.Client()

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
parameterized_query = """
SELECT
    fullVisitorId,
    channelGrouping,
    prod.productListName,
    trafficSource.referralPath,
    trafficSource.campaign,
    trafficSource.keyword,
    device.operatingSystem,
    visitNumber,
    date,
    prod.productListPosition,
    hits.eCommerceAction.action_type,
    prod.localProductPrice,
    prod.v2ProductCategory,
    prod.v2ProductName,
    prod.productVariant,
    hits.latencyTracking.pageLoadTime,
    hits.type,
    hits.hour,
    trafficSource.medium,
    trafficSource.source,
    device.deviceCategory,
    device.browser,
    geoNetwork.country
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(hits) AS hits, 
    UNNEST(hits.product) AS prod
WHERE
    _TABLE_SUFFIX BETWEEN '%s' AND '%s'
"""
start_date = '20160801'
end_date = '20160804'
# end_date = '20170801'
parameterized_query_df = bigquery_client.query((parameterized_query % (start_date, end_date))).to_dataframe()
parameterized_query_df

,fullVisitorId,channelGrouping,productListName,referralPath,campaign,keyword,operatingSystem,visitNumber,date,productListPosition,...,v2ProductName,productVariant,pageLoadTime,type,hour,medium,source,deviceCategory,browser,country
0,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,1,...,Red Spiral Google Notebook,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
1,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,2,...,16 oz. Hot/Cold Tumbler,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
2,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,3,...,Google Trucker Hat,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
3,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,4,...,Mistral Rucksack,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
4,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,5,...,Oasis Backpack,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
5,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,6,...,Deluge Waterproof Backpack,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
6,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,7,...,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
7,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,8,...,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
8,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,9,...,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway
9,5320937337095104430,Direct,Category,None,(not set),None,Android,2,20160802,10,...,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),NaN,PAGE,1,(none),(direct),mobile,Chrome,Norway


In [7]:
import datetime
parameterized_query_df['date'] = pd.to_datetime(parameterized_query_df['date'])
parameterized_query_df['month'] = parameterized_query_df['date'].dt.month
parameterized_query_df['day'] = parameterized_query_df['date'].dt.weekday

In [8]:
parameterized_query_df=parameterized_query_df.set_index('fullVisitorId')

In [9]:
df = parameterized_query_df.copy()

In [10]:
nacol = pd.DataFrame(df.isna().any()).reset_index()
nacol.columns = ['column', 'Bool']
nacol = nacol[nacol.Bool == True]
nacol

,column,Bool
2,referralPath,True
4,keyword,True
14,pageLoadTime,True


In [27]:
for i in nacol.column:
    try:
        df[i] = np.where(np.isnan(df[i]),df[i].median(), df[i])
    except:
        df[i] = np.where(df[i].isna(),"No", df[i])

# Label Encoding

In [28]:
from sklearn import preprocessing

In [29]:
le = preprocessing.LabelEncoder()

In [30]:
pd.set_option('display.max_columns', None)
df.head()

,channelGrouping,productListName,referralPath,campaign,keyword,operatingSystem,visitNumber,date,productListPosition,action_type,localProductPrice,v2ProductCategory,v2ProductName,productVariant,pageLoadTime,type,hour,medium,source,deviceCategory,browser,country,month,day
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,
5320937337095104430,1,1,55.0,0,12.0,1,2,2016-08-02,1,0,9990000,34,209,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55.0,0,12.0,1,2,2016-08-02,2,0,10990000,34,1,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55.0,0,12.0,1,2,2016-08-02,3,0,28990000,34,120,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55.0,0,12.0,1,2,2016-08-02,4,0,69990000,34,195,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55.0,0,12.0,1,2,2016-08-02,5,0,99990000,34,196,23,No,1,1,0,0,1,2,60,8,1


In [31]:
le.fit(df['type'])
df['type']=le.transform(df['type'])
le.fit(df['medium'])
df['medium']=le.transform(df['medium'])
le.fit(df['deviceCategory'])
df['deviceCategory']=le.transform(df['deviceCategory'])
le.fit(df['browser'])
df['browser'] = le.transform(df['browser'])
le.fit(df['country'])
df['country'] = le.transform(df['country'])
le.fit(df['source'])
df['source'] = le.transform(df['source'])
le.fit(df['v2ProductCategory'])
df['v2ProductCategory'] = le.transform(df['v2ProductCategory'])
le.fit(df['v2ProductName'])
df['v2ProductName'] = le.transform(df['v2ProductName'])
le.fit(df['channelGrouping'])
df['channelGrouping'] = le.transform(df['channelGrouping'])
le.fit(df['referralPath'])
df['referralPath'] = le.transform(df['referralPath'])
le.fit(df['campaign'])
df['campaign'] = le.transform(df['campaign'])
le.fit(df['keyword'])
df['keyword'] = le.transform(df['keyword'])
le.fit(df['operatingSystem'])
df['operatingSystem'] = le.transform(df['operatingSystem'])
le.fit(df['productListName'])
df['productListName'] = le.transform(df['productListName'])
le.fit(df['productVariant'])
df['productVariant'] = le.transform(df['productVariant'])

In [32]:
pd.set_option('display.max_columns', None)
df.head()

,channelGrouping,productListName,referralPath,campaign,keyword,operatingSystem,visitNumber,date,productListPosition,action_type,localProductPrice,v2ProductCategory,v2ProductName,productVariant,pageLoadTime,type,hour,medium,source,deviceCategory,browser,country,month,day
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,
5320937337095104430,1,1,55,0,12,1,2,2016-08-02,1,0,9990000,34,209,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,2016-08-02,2,0,10990000,34,1,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,2016-08-02,3,0,28990000,34,120,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,2016-08-02,4,0,69990000,34,195,23,No,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,2016-08-02,5,0,99990000,34,196,23,No,1,1,0,0,1,2,60,8,1


In [39]:
df['pageLoadTime'] = np.where(df.pageLoadTime=='No',0, df.pageLoadTime)

In [40]:
df['pageLoadTime'].unique()

array([0, 4063.0, 4469.0, 2367.0, 13658.0, 1836.0, 13242.0, 1476.0, 979.0,
       2577.0, 1855.0, 1572.0, 1417.0, 1874.0, 1230.0, 2668.0, 1406.0,
       1415.0, 1994.0, 1604.0, 2390.0, 2936.0, 4129.0, 11973.0, 1208.0,
       3008.0, 1011.0, 1456.0, 5691.0, 1730.0, 534.0, 6776.0, 4157.0,
       1643.0, 3256.0, 1607.0, 1667.0, 1790.0, 1822.0, 1485.0, 2657.0,
       3314.0, 3080.0, 3275.0, 4258.0, 1809.0, 3452.0, 1885.0, 1109.0,
       1255.0, 1548.0, 1082.0, 1584.0, 910.0, 1279.0, 1043.0, 1605.0],
      dtype=object)

## Dummifying Variables

In [41]:
dum1 = pd.get_dummies(parameterized_query_df['action_type'], prefix='action_type', prefix_sep='__')

In [42]:
print('dum1', dum1.columns)

dum1 Index(['action_type__0', 'action_type__1', 'action_type__2', 'action_type__3',
       'action_type__4', 'action_type__5', 'action_type__6'],
      dtype='object')


In [43]:
goal = dum1['action_type__6']

In [44]:
df = df.drop('date', axis=1)

In [45]:
df = df.drop('action_type', axis=1)

In [46]:
df.isnull().sum()

channelGrouping        0
productListName        0
referralPath           0
campaign               0
keyword                0
operatingSystem        0
visitNumber            0
productListPosition    0
localProductPrice      0
v2ProductCategory      0
v2ProductName          0
productVariant         0
pageLoadTime           0
type                   0
hour                   0
medium                 0
source                 0
deviceCategory         0
browser                0
country                0
month                  0
day                    0
dtype: int64

In [47]:
df.head()

,channelGrouping,productListName,referralPath,campaign,keyword,operatingSystem,visitNumber,productListPosition,localProductPrice,v2ProductCategory,v2ProductName,productVariant,pageLoadTime,type,hour,medium,source,deviceCategory,browser,country,month,day
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,
5320937337095104430,1,1,55,0,12,1,2,1,9990000,34,209,23,0,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,2,10990000,34,1,23,0,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,3,28990000,34,120,23,0,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,4,69990000,34,195,23,0,1,1,0,0,1,2,60,8,1
5320937337095104430,1,1,55,0,12,1,2,5,99990000,34,196,23,0,1,1,0,0,1,2,60,8,1


In [48]:
df.shape

(448605, 22)

### Split Data

In [49]:
from sklearn.model_selection import train_test_split
y=goal
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)

In [50]:
sum(y_test>0)

133

# Random Forest

In [51]:
from sklearn import ensemble

In [52]:
randomForest = ensemble.RandomForestClassifier()

In [53]:
randomForest.set_params(n_estimators=50, random_state=42, max_features=10, oob_score=True)
model = randomForest.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("The training error of random forest is: %.5f" %(1 - randomForest.score(X_train, y_train)))
print("The test     error of random forest is: %.5f" %(1 - randomForest.score(X_test, y_test)))

The training error of random forest is: 0.00042
The test     error of random forest is: 0.00059


In [54]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

Accuracy Score : 0.9994129972804684
Precision Score : 0.8068181818181818
F1 Score : 0.6425339366515838


In [55]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

Confusion Matrix : 
[[134432     17]
 [    62     71]]


In [56]:
feature_importance = randomForest.feature_importances_
print([t[1] for t in zip(feature_importance, X_train.columns) if t[0]>0])
feature_importance

['channelGrouping', 'productListName', 'referralPath', 'campaign', 'keyword', 'operatingSystem', 'visitNumber', 'productListPosition', 'localProductPrice', 'v2ProductCategory', 'v2ProductName', 'productVariant', 'type', 'hour', 'medium', 'source', 'deviceCategory', 'browser', 'country', 'day']


array([2.30949587e-02, 1.43146433e-02, 1.63810160e-02, 8.08240127e-06,
       5.81344334e-03, 4.35584412e-02, 7.61938563e-02, 5.74467755e-03,
       2.66542698e-01, 1.99481010e-02, 1.12191137e-01, 1.20785394e-01,
       0.00000000e+00, 1.30209840e-02, 1.35262230e-01, 1.20936151e-02,
       1.43099781e-02, 4.00598626e-03, 2.23836980e-03, 1.20719523e-02,
       0.00000000e+00, 1.02420436e-01])

In [57]:
imp=zip(X_train.columns, feature_importance)
res = sorted(imp, key=lambda x: x[1], reverse=True)
res

[('localProductPrice', 0.2665426982119951),
 ('hour', 0.13526222970080787),
 ('productVariant', 0.12078539443635407),
 ('v2ProductName', 0.11219113686307063),
 ('day', 0.10242043550949508),
 ('visitNumber', 0.07619385631986271),
 ('operatingSystem', 0.04355844121509815),
 ('channelGrouping', 0.023094958699087346),
 ('v2ProductCategory', 0.019948100995792615),
 ('referralPath', 0.016381015962650035),
 ('productListName', 0.014314643250375932),
 ('source', 0.014309978113529086),
 ('type', 0.01302098399357612),
 ('medium', 0.012093615090748553),
 ('country', 0.012071952287994426),
 ('keyword', 0.005813443342643815),
 ('productListPosition', 0.00574467754643258),
 ('deviceCategory', 0.004005986260264827),
 ('browser', 0.00223836979895339),
 ('campaign', 8.08240126774588e-06),
 ('pageLoadTime', 0.0),
 ('month', 0.0)]

In [58]:
x = model.predict_proba(X_train)
x = x[:,1]

In [59]:
pd.Series(x).value_counts()

0.000000    313099
0.020000       123
0.344573        39
0.010000        29
1.000000        20
0.040000        19
0.980000        17
0.060000        16
0.006667        16
0.080000        13
0.025000        11
0.005000        10
0.124063         7
0.336858         7
0.900000         7
0.880000         7
0.117700         7
0.289473         7
0.191405         6
0.304130         6
0.100000         6
0.153398         6
0.180582         6
0.313378         6
0.160000         6
0.126040         6
0.820000         6
0.860000         6
0.192385         6
0.960000         6
             ...  
0.023016         1
0.018571         1
0.076667         1
0.156353         1
0.102381         1
0.008000         1
0.035000         1
0.116389         1
0.022000         1
0.943333         1
0.028000         1
0.004737         1
0.600000         1
0.256333         1
0.082857         1
0.237714         1
0.081778         1
0.866667         1
0.016667         1
0.788571         1
0.039333         1
0.856667    

## Adding Add to Cart to Model and Run Random Forest

In [60]:
df2 = df.copy()

In [61]:
df2 = pd.concat([df2, dum1['action_type__3']], axis=1)

In [62]:
df2 = pd.concat([df2, dum1['action_type__1']], axis=1)

In [63]:
df2 = pd.concat([df2, dum1['action_type__2']], axis=1)

In [64]:
df2.head()

,channelGrouping,productListName,referralPath,campaign,keyword,operatingSystem,visitNumber,productListPosition,localProductPrice,v2ProductCategory,v2ProductName,productVariant,pageLoadTime,type,hour,medium,source,deviceCategory,browser,country,month,day,action_type__3,action_type__1,action_type__2
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,,
5320937337095104430,1,1,55,0,12,1,2,1,9990000,34,209,23,0,1,1,0,0,1,2,60,8,1,0,0,0
5320937337095104430,1,1,55,0,12,1,2,2,10990000,34,1,23,0,1,1,0,0,1,2,60,8,1,0,0,0
5320937337095104430,1,1,55,0,12,1,2,3,28990000,34,120,23,0,1,1,0,0,1,2,60,8,1,0,0,0
5320937337095104430,1,1,55,0,12,1,2,4,69990000,34,195,23,0,1,1,0,0,1,2,60,8,1,0,0,0
5320937337095104430,1,1,55,0,12,1,2,5,99990000,34,196,23,0,1,1,0,0,1,2,60,8,1,0,0,0


In [65]:
from sklearn.model_selection import train_test_split
y2=goal
X_train2, X_test2, y_train2, y_test2 = train_test_split(df2, y2, test_size=0.3, random_state=42)

In [66]:
randomForest2 = ensemble.RandomForestClassifier()

In [67]:
randomForest2.set_params(n_estimators=50, random_state=42, max_features=10, oob_score=True)
model2 = randomForest2.fit(X_train2, y_train2)
y_pred2 = model2.predict(X_test2)
print("The training error of random forest is: %.5f" %(1 - randomForest2.score(X_train2, y_train2)))
print("The test     error of random forest is: %.5f" %(1 - randomForest2.score(X_test2, y_test2)))

The training error of random forest is: 0.00042
The test     error of random forest is: 0.00061


In [68]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test2,y_pred2)))
print('Precision Score : ' + str(precision_score(y_test2,y_pred2)))
print('F1 Score : ' + str(f1_score(y_test2,y_pred2)))

Accuracy Score : 0.9993907060379545
Precision Score : 0.7684210526315789
F1 Score : 0.6403508771929824


In [69]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test2,y_pred2)))

Confusion Matrix : 
[[134427     22]
 [    60     73]]


In [70]:
feature_importance2 = randomForest2.feature_importances_
print([t[1] for t in zip(feature_importance2, X_train2.columns) if t[0]>0])
feature_importance2

['channelGrouping', 'productListName', 'referralPath', 'keyword', 'operatingSystem', 'visitNumber', 'productListPosition', 'localProductPrice', 'v2ProductCategory', 'v2ProductName', 'productVariant', 'type', 'hour', 'medium', 'source', 'deviceCategory', 'browser', 'country', 'day', 'action_type__3', 'action_type__1', 'action_type__2']


array([2.16646202e-02, 1.56281893e-02, 1.56379783e-02, 0.00000000e+00,
       5.04400007e-03, 4.20658222e-02, 7.24303890e-02, 1.66730564e-02,
       2.67409648e-01, 2.52275362e-02, 1.14420342e-01, 1.07554676e-01,
       0.00000000e+00, 1.87953607e-02, 1.37861874e-01, 1.11791332e-02,
       1.13016044e-02, 4.13714861e-03, 3.21903245e-03, 1.04199025e-02,
       0.00000000e+00, 9.40281615e-02, 4.48289632e-03, 7.65898952e-04,
       5.27304894e-05])

In [71]:
imp2 = zip(X_train2.columns, feature_importance2)
res2 = sorted(imp2, key=lambda x: x[1], reverse=True)
res2

[('localProductPrice', 0.26740964772687215),
 ('hour', 0.13786187442368805),
 ('v2ProductName', 0.11442034152157637),
 ('productVariant', 0.1075546756436758),
 ('day', 0.09402816147497305),
 ('visitNumber', 0.07243038897205306),
 ('operatingSystem', 0.04206582218911875),
 ('v2ProductCategory', 0.02522753623892022),
 ('channelGrouping', 0.021664620152022453),
 ('type', 0.01879536068953755),
 ('productListPosition', 0.016673056428122463),
 ('referralPath', 0.015637978283667874),
 ('productListName', 0.01562818933427606),
 ('source', 0.011301604384694061),
 ('medium', 0.011179133167787268),
 ('country', 0.010419902474236293),
 ('keyword', 0.0050440000660583505),
 ('action_type__3', 0.004482896324428324),
 ('deviceCategory', 0.004137148609466529),
 ('browser', 0.003219032453587855),
 ('action_type__1', 0.0007658989518532959),
 ('action_type__2', 5.2730489384186744e-05),
 ('campaign', 0.0),
 ('pageLoadTime', 0.0),
 ('month', 0.0)]

In [72]:
x2 = model.predict_proba(X_train)
x2 = x2[:,1]

In [73]:
pd.Series(x2).value_counts()

0.000000    313099
0.020000       123
0.344573        39
0.010000        29
1.000000        20
0.040000        19
0.980000        17
0.060000        16
0.006667        16
0.080000        13
0.025000        11
0.005000        10
0.124063         7
0.336858         7
0.900000         7
0.880000         7
0.117700         7
0.289473         7
0.191405         6
0.304130         6
0.100000         6
0.153398         6
0.180582         6
0.313378         6
0.160000         6
0.126040         6
0.820000         6
0.860000         6
0.192385         6
0.960000         6
             ...  
0.023016         1
0.018571         1
0.076667         1
0.156353         1
0.102381         1
0.008000         1
0.035000         1
0.116389         1
0.022000         1
0.943333         1
0.028000         1
0.004737         1
0.600000         1
0.256333         1
0.082857         1
0.237714         1
0.081778         1
0.866667         1
0.016667         1
0.788571         1
0.039333         1
0.856667    

In [75]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
rfgs_acc = accuracy_score(y_test2,y_pred2)
rfgs_pscore = precision_score(y_test2,y_pred2) 
rfgs_recall = recall_score(y_test2,y_pred2) 
rf_f1 = f1_score(y_test2,y_pred2)

In [76]:
scores = pd.DataFrame({'Type':['Accuracy', 'Precision', 'Recall', 'F1'], 'Random Forest':[rfgs_acc, rfgs_pscore, rfgs_recall, rf_f1]})
                   #   "GB": [gb_acc, gb_pscore, gb_recall, gb_f1]})
#scores = pd.melt(scores, id_vars = "Type", value_vars=['Random Forest', 'GB'])
scores = pd.melt(scores, id_vars = "Type", value_vars=['Random Forest'])
scores.columns = ["Type", "Model", "Score"]
print(scores)
goog = ["#4285F4", "#DB4437", "#F4B400", "#0F9D58"]
sns.set(style="whitegrid")
sns.despine()
sns.barplot(x="Score", y="Type", data=scores,  palette = goog)
plt.savefig("rf_scores.png")

        Type          Model     Score
0   Accuracy  Random Forest  0.999391
1  Precision  Random Forest  0.768421
2     Recall  Random Forest  0.548872
3         F1  Random Forest  0.640351


NameError: name 'sns' is not defined